# 목적 (Goal)

## 프로젝트 목적
시군구 단위 의료 취약도를 정량화하여 취약 지역을 식별하고, 격차의 원인(인구·기관·의사/간호 인력·고령화)을 규명해 의료 취약지역을 확인한다.  

- 병원/인력 공급 데이터를 수집하여 사용한다.  
- 시군구별 기관·인력 규모의 분포/편차, 의사·간호 균형/상관 등을 통해 공급 과소 지역을 파악한다.  
- 수요(인구/고령화) 미반영이라는 한계를 명시하고, 다음 단계에서 수요 데이터를 추가 수집이 필요함을 확인한다.  

> 해당 프로젝트는 다음 머신러닝 프로젝트에 사용할 내용까지 고려하여 데이터를 수집할 필요가 있습니다.  
> 데이터 수집 후 이후 머신러닝을 통해 진행할 수 있는 태스크까지 고려해주세요.

## 1) 데이터 수집

**보건복지부 공공데이터 API**를 활용하여 전국 병원/의원 정보를 수집합니다.  

### 절차
1. API 요청: [보건복지부 공공데이터](https://www.data.go.kr/data/15001698/openapi.do) 사이트의 데이터를 인증키를 사용하여 페이지 단위로 가져옵니다.  
2. 전체 데이터 건수 확인: `extract_total_count()` 로 몇 건이 있는지 확인합니다.  
3. 데이터 아이템 추출: `extract_items()` 로 실제 레코드만 꺼냅니다.  
4. 반복 수집: 전체 페이지를 순회하며 데이터를 모두 모읍니다.  
5. CSV 저장: `write_csv()` 로 수집한 데이터를 파일로 정리합니다.  

> 최종적으로 **hospitals_all.csv** 파일이 생성되며, 이후 분석 단계에서 사용합니다.

In [ ]:
import os
import time
import math
import csv
import requests
from dotenv import load_dotenv

load_dotenv()

API_URL = "http://apis.data.go.kr/B551182/hospInfoServicev2/getHospBasisList"
API_KEY = os.getenv("API_KEY")

OUT_CSV = "hospitals_all.csv"
NUM_ROWS = 1000          # API가 허용하는 최대치로 조정 가능
MAX_RETRIES = 3
RETRY_WAIT = 1.5

def request_json(page_no, num_rows):
    # JSON 응답을 얻기 위해 resultType/_type/type 3가지를 순차 시도
    base_params = {
        "serviceKey": API_KEY,
        "pageNo": page_no,
        "numOfRows": num_rows,
    }
    # 시도 순서: resultType, _type, type
    trial_params = [
        {**base_params, "resultType": "json"},
        {**base_params, "_type": "json"},
        {**base_params, "type": "json"},
    ]

    last_exc = None
    for params in trial_params:
        for attempt in range(1, MAX_RETRIES+1):
            try:
                r = requests.get(API_URL, params=params, timeout=20)
                r.raise_for_status()
                data = r.json()   # JSON 파싱 시도
                return data
            except Exception as e:
                last_exc = e
                if attempt < MAX_RETRIES:
                    time.sleep(RETRY_WAIT)
                else:
                    # 다음 파라미터 키 조합으로 넘어감
                    pass
    # 세 가지 모두 실패
    raise RuntimeError(f"JSON 요청/파싱 실패: {last_exc}")

def extract_total_count(resp_json):
    # v2 형식(보건복지부 계열) 응답에서 totalCount를 다양한 케이스로 안전 파싱.
    # 보편적 구조: {'response': {'body': {'totalCount': N, 'items': {'item': [...]}}}}
    return int(
        resp_json["response"]["body"]["totalCount"]
    )

def extract_items(resp_json):
    # items 배열을 안전하게 추출.
    items = resp_json["response"]["body"]["items"]["item"]
    if items is None:
        return []
    if isinstance(items, list):
        return items
    return [items]

def write_csv(rows, path):
    # 모든 키 수집 (컬럼 공일)
    fieldnames = []
    for row in rows:
        for k in row.keys():
            if k not in fieldnames:
                fieldnames.append(k)

    with open(path, "w", newline="", encoding="utf-8-sig") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames, extrasaction="ignore")
        writer.writeheader()
        for row in rows:
            writer.writerow({k: row.get(k, "") for k in fieldnames})

def main():
    # 첫 페이지 totalCount 확인
    first = request_json(page_no=1, num_rows=NUM_ROWS)
    total_count = extract_total_count(first)
    items = extract_items(first)
    print(f"[초정보] totalCount={total_count}, 첫 페이지 수집={len(items)}")

    if total_count == 0:
        write_csv([], OUT_CSV)
        print(f"CSV생성: {OUT_CSV}")
        return

    total_pages = max(1, math.ceil(total_count / NUM_ROWS))

    # 나머지 페이지 순회
    all_rows = []
    all_rows.extend(items)

    for page in range(2, total_pages + 1):
        resp = request_json(page_no=page, num_rows=NUM_ROWS)
        page_items = extract_items(resp)
        all_rows.extend(page_items)
        print(f"[수집현황] {page}/{total_pages} 수집: {len(page_items)}건 (누적 {len(all_rows)}/{total_count})")
        time.sleep(0.15)

    # CSV 저장
    write_csv(all_rows, OUT_CSV)
    save_dir = os.path.abspath(os.path.join(os.getcwd(), "data"))
    os.makedirs(save_dir, exist_ok=True)   # data 디렉토리가 없으면 생성

    # 최종 저장 경로
    save_path = os.path.join(save_dir, OUT_CSV)

    print(f"총 {len(all_rows)}건을 CSV로 저장: {save_path}")


if __name__ == "__main__":
    main()

## 병원 데이터 (hospital_all.csv) 확인

- **행/열:** 79,256 × 30 → 전국 단위 병원·의원·보건소 정보  

### 주요 변수
- `addr`: 주소 (약 7만6천 개 고유값, 결측치 없음)  
- `clCd`, `clCdNm`: 병원종별 코드와 명칭  
- `XPos`, `YPos`: 좌표값  
- `ykiho`: 의료기관 고유코드  
- `drTotCnt`: 전체 의사 수 (결측치 있음, 분포 편차 큼)  
- `estbDd`: 설립일 (YYYYMMDD 정수, 결측 있음)  
- `hospUrl`: 병원 웹사이트 주소 (7만 건 이상 결측 → 분석 불필요)  
- `cmdc*`, `dety*`, `mdept*`: 전공의·인턴·레지던트·전임의 수 (대부분 0, 소수 병원만 값 존재)  

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import re
import os

In [ ]:
base_dir = os.path.abspath(os.path.join(os.getcwd(), "data"))
file_path = os.path.join(base_dir, "hospitals_all.csv")

df = pd.read_csv(file_path)

# 탐색적 데이터 분석 (EDA)

- 목적은 데이터의 특징을 직관적으로 이해하고, 문제 정의와 연결할 인사이트를 도출하는 것입니다.

## 데이터 전처리
원시 데이터를 분석 가능한 상태로 다듬는 과정입니다.

## 절차
1. 결측치 처리  
   - `fillna(값)` : 평균, 중앙값으로 대체  
   - `dropna()` : 행 또는 열 삭제  

2. 이상치 처리  
   - IQR 방법: 사분위수 범위 밖의 값을 제거  
   - 단순 조건식: `df[df["col"] < threshold]`  

3. 자료형 변환  
   - 날짜형: `pd.to_datetime(df["estbDd"])`  
   - 숫자형: `astype(int/float)`  

4. 파생 변수 생성  
   - 날짜 → 연/월/요일  
   - 의사수 대비 병상수 비율 등 지표 만들기  

## 필요한 방법
- Pandas 내장 메서드(`fillna`, `dropna`, `astype`, `to_datetime`)  
- 조건 필터링(`df[col] > value`)  
- 새 컬럼 추가(`df["new"] = ...`)  

전처리를 통해 데이터 품질을 확보해야 신뢰할 수 있는 분석 결과를 얻을 수 있습니다.

In [ ]:
print("info()")
print(df.info())

print("\n결측치 수")
na = df.isna().sum().sort_values(ascending=False)
print(na.head(20))

dup_cnt = df.duplicated().sum()
print(f"\n중복 행 수: {dup_cnt:,}")

In [ ]:
def clean_col(c):
    c = str(c).replace("\n", " ").replace("\t", " ")
    c = re.sub(r"\s+", " ", c).strip()
    return c

df.columns = [clean_col(c) for c in df.columns]
norm_map = {c: re.sub(r"\W+", "_", c.lower()).strip("_") for c in df.columns}
df.rename(columns=norm_map, inplace=True)

df.head(2)

In [ ]:
# 타입 정제 (날짜/숫자 자동 캐스팅)
# 'date(일자|등록일|개설일)' 포함 컬럼 = 날짜 변환
# 숫자로 추정될 수치 변환
# df.columns에서 컬럼명을 훑어서, "date", "dt", "일자", "날짜", "개설", "등록" 같은 패턴이 들어가면 날짜 관련 컬럼이라고 판단
# 문자열로 저장돼 있지만 실제 datetime 타입으로 바꿔주는 부분
date_like = [c for c in df.columns if re.search(r"(date|dt|일자|날짜|개설|등록)", c, re.I)]
for c in date_like:
    df[c] = pd.to_datetime(df[c], errors="coerce")

# 문자열을 자동으로 숫자형(int/float)으로 바꿔주기
for c in df.columns:
    if df[c].dtype == "object":
        s = df[c].astype(str).str.replace(",", "").str.strip()
        # 숫자로 보이는 비율이 절반 이상이면 숫자화 시도
        if s.str.match(r"^-?\d+(\.\d+)?$").mean() > 0.5:
            df[c] = pd.to_numeric(s, errors="coerce")

print(df.dtypes.value_counts())

In [ ]:
# 데이터프레임 컬럼을 타입별로 분류하고 기초 특성을 확인하기 위한 코드
num_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]
obj_cols = [c for c in df.columns if pd.api.types.is_object_dtype(df[c])]
dt_cols  = [c for c in df.columns if pd.api.types.is_datetime64_any_dtype(df[c])]

print(f"수치형: {len(num_cols)}, 범주형: {len(obj_cols)}, 날짜형: {len(dt_cols)}")

pd.options.display.float_format = "{:.3f}".format

if num_cols:
    display(df[num_cols].describe().T)

cat_card = pd.Series({c: df[c].nunique(dropna=True) for c in obj_cols}).sort_values()
display(cat_card.to_frame("nunique").sort_values("nunique"))

# 탐색적 데이터 분석 (EDA)

- 목적: EDA는 데이터를 요약, 시각화하여 패턴과 인사이트를 발견하는 단계입니다.

## 절차

1. **기초 통계 확인**
   - `df.describe()` 로 평균, 표준편차, 최소값/최댓값 확인

2. **분포 확인**
   - 히스토그램: `df["병상수"].hist()`
   - 박스플롯: 이상치 탐지

3. **범주형 변수 빈도**
   - `value_counts().head()`
   - 막대그래프로 상위 항목 확인

4. **관계 분석**
   - 산점도: 수치형 변수 간 관계 (`sns.scatterplot`)
   - 그룹별 비교: 지역/요일별 평균값

## 필요한 방법

- **Matplotlib / Seaborn 시각화**
  - `plt.hist`, `sns.countplot`, `sns.scatterplot`, `plt.plot`
- **한 줄 해석 작성**
  - 단순 그래프가 아니라, “이 값이 높다/특정 패턴이 반복된다” 등 설명 추가

목표는 **데이터의 특징을 직관적으로 이해하고, 문제 정의와 연결할 인사이트를 도출하는 것**입니다.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

plt.rcParams['font.family'] = 'Malgun Gothic'
plt.style.use("ggplot")  # 가독성 좋은 스타일

# 변수 설명 매핑
col_desc = {
    "clcd": "병원종별 코드",
    "clcdnm": "병원종별 이름",
    "drtotcnt": "전체 의사 수",
    "pnurscnt": "조산사 수",
    "cmdcgrdrcnt": "일반의 수",
    "cmdcntnrcnt": "인턴 수",
    "cmdresdntcnt": "레지던트 수",
    "cmdsdrcnt": "전문의 수",
    "detygdrdcnt": "공중보건의 일반의 수",
    "detyinterncnt": "공중보건의 인턴 수",
    "detyresdntcnt": "공중보건의 레지던트 수",
    "detysdrcnt": "공중보건의 전문의 수",
    "mdeptgdrdcnt": "진료과 일반의 수",
    "mdeptinterncnt": "진료과 인턴 수",
    "mdeptresdntcnt": "진료과 레지던트 수",
    "mdeptsdrnt": "진료과 전문의 수",
    "xpos": "병원 위치 X좌표",
    "ypos": "병원 위치 Y좌표",
    "estbdd": "설립 일자(YYYYMMDD)",
    "addr": "주소",
    "postno": "우편번호",
    "sidoCdNm": "시도명"
}

# 1. 수치형: 의사 수 분포
plt.figure(figsize=(8,5))
sns.histplot(df["drtotcnt"], bins=50, kde=False)
plt.yscale("log")  # y축 로그 스케일
plt.xscale("log")  # x축 로그 스케일
plt.title("병원별 의사 수 분포 (로그 스케일)")
plt.xlabel("의사 수")
plt.ylabel("병원 수 (로그 스케일)")
plt.show()

# 2. 범주형: 시도별 병원 수
plt.figure(figsize=(10,5))
vc = df["sidocdnm"].value_counts()
sns.barplot(x=vc.index, y=vc.values, palette="Blues_r")
plt.title("시도별 병원 수", fontsize=14)
plt.xlabel("시도명"); plt.ylabel("병원 수")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

# 3. 범주형: 병원명 분포
plt.figure(figsize=(10,5))
vc = df["clcdnm"].value_counts()
sns.barplot(x=vc.index, y=vc.values, palette="Greens_r")
plt.title("병원종별 분포", fontsize=14)
plt.xlabel("병원종별 이름"); plt.ylabel("병원 수")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

# 4. 대형병원의 분포 확인
# 종합병원 or 상급종합, 의사 수가 100명 이상
large_hospitals = df[(df["clcdnm"].isin(["종합병원", "상급종합"])) & (df["drtotcnt"] >= 100)]
vc = large_hospitals["sggucdnm"].value_counts()

plt.figure(figsize=(12,6))
sns.barplot(x=vc.index, y=vc.values, palette="Reds_r")
plt.title("시군구별 대형병원 수 (종합병원·상급종합, 의사 100명 이상)", fontsize=14)
plt.xlabel("시군구명"); plt.ylabel("대형병원 수")
plt.xticks(rotation=90, ha="right")
plt.tight_layout()
plt.show()

# 5. 시도별 대형병원 수
vc = large_hospitals["sidocdnm"].value_counts()
vc_low10 = vc.sort_values(ascending=True).head(10)

plt.figure(figsize=(12,6))
sns.barplot(x=vc_low10.index, y=vc_low10.values, palette="Reds_r")
plt.title("대형병원이 적은 시도 TOP 10 (종합병원·상급종합)", fontsize=14)
plt.xlabel("시도명"); plt.ylabel("대형병원 수")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

num_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]

if num_cols:
    # 1. 상관계수 계산
    corr = df[num_cols].corr(method="pearson")
    print("상관계수 매트릭스")
    display(corr.round(2))   # 소수점 2자리까지만 표시

    # 2. 히트맵 시각화 (Seaborn 활용)
    plt.figure(figsize=(12,10))
    sns.heatmap(
        corr,
        cmap="coolwarm",      # 색상: 파랑(음수) ↔ 빨강(양수)
        center=0,             # 0을 기준으로 색상 나눔
        annot=True,           # 셀 안에 상관계수 숫자 표시
        fmt=".2f",            # 소수점 2자리
        linewidths=0.5,       # 경계선 표시
        cbar_kws={"label": "Correlation"}  # 컬러바 제목
    )

    plt.title("수치형 변수 상관관계 히트맵", fontsize=16)
    plt.tight_layout()
    plt.show()

## 1. 의사 관련 변수들의 강한 상관
- `drTotCnt (전체 의사 수)` ↔ `mdeptintncnt`, `mdeptresdntcnt`, `mdeptsdrncnt` : **0.9 이상**
  - 사실상 같은 개념을 다른 변수로 나눈 것 → **중복 정보 존재**

---

## 2. 인턴·레지던트 계열의 자기상관
- `cmdcintncnt`, `cmdcresdntcnt`, `cmdcsdrncnt` : **0.7 이상**
  - 역시 동일 범주(수련의)를 세분화 → **중복 정보 존재**

---

## 3. 행정구역 코드 및 좌표
- `sgguCd`, `sidodCd` : 서로 완전히 중복 (1.0)
- `postno` ↔ `xpos / ypos` : 음의 상관 (-0.9대) → 사실상 좌표와 같은 역할
- 그대로 쓰기보다는 **지역 집계 후 활용**
- 공급 변수들은 **중복성이 매우 높음**
  - 핵심 변수는 `drTotCnt`(의사 수), `병원 수` 정도
  - 행정구역/좌표는 식별자 또는 지역 단위 집계용으로 활용

In [ ]:
plt.figure(figsize=(6,8))
plt.scatter(df["xpos"], df["ypos"], s=1, alpha=0.3)
plt.title("전국 병원 위치 분포")
plt.xlabel("X 좌표")
plt.ylabel("Y 좌표")
plt.show()

In [ ]:
top_doctors = df.groupby("sggucdnm")["drtotcnt"].sum().sort_values(ascending=False).tail(10)
top_doctors.plot(kind="bar", figsize=(10,5))
plt.title("시군구별 의사 수 하위 10")
plt.ylabel("의사 수")
plt.show()

In [ ]:
top_hospitals_sido = df.groupby("sidocdnm")["ykiho"].count().sort_values(ascending=False).head(10)
plt.figure(figsize=(8,4))
top_hospitals_sido.plot(kind="bar", color="darkcyan")
plt.title("시도별 병원 수 Top 10")
plt.ylabel("병원 수")
plt.xlabel("시도명")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

top_hospitals_sido = df.groupby("sidocdnm")["ykiho"].count().sort_values(ascending=False).tail(10)
plt.figure(figsize=(8,4))
top_hospitals_sido.plot(kind="bar", color="darkcyan")
plt.title("시도별 병원 수 하위 10")
plt.ylabel("병원 수")
plt.xlabel("시도명")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

# 결론
- 시군구별 기관·인력 지표의 분포와 편차가 확인되었다.
- 공간적으로 과소 지역의 패턴이 드러난다.
- 좌표/행정구역 품질이 양호해 지도 기반 시각화도 고려하여 의사결정이 가능하다.

# 한계
- 현재 결과는 공급 지표에 국한되어 있어, 실제 부족/취약의 정도를 판단하기엔 불충분하다.
- 같은 공급 수준이라도 수요 규모·구조가 다르면 체감 접근성·취약도는 달라질 수 있다.

# 결정 및 다음 단계
1. 수요 측정치를 정의·추가 수집하여 시군구 단위로 결합한다.
   - 예: 인구 규모, 연령구조/고령자 비율 (가능 시) 인구 변화 추세 등

2. 결합 후 공급-수요 결합 지표를 실제 산출한다.
   - 예: 인구당 의사, 연령 가중 지수

3. 결합 지표를 랭킹/지도/분위수로 제시해 취약 상위 지역을 식별하고, 원인 변수의 기여도를 평가한다.

> 요약: 공급 격차는 명확하다. 수요를 결합하여 확인하면 취약도를 더 명확히 분석하고 이후에 취약한지 얼마나 예측도 가능할 것으로 보인다.